In [1]:
import random
import time
import os
import sys

import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

sys.path.append('../pyfiles')
from Models import AFM, DeepAFM, ADFM

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)


1 Physical GPUs, 1 Logical GPUs


In [6]:
df = pd.read_csv('../data/frappe_all.csv', usecols=['user', 'item','daytime', 'weekday', 'isweekend', 'homework', 'cost',
       'weather', 'country', 'city', 'label'])

In [7]:
df

,user,item,daytime,weekday,isweekend,homework,cost,weather,country,city,label
0,66,2471,morning,sunday,weekend,unknown,free,rainy,United States,0,1
1,269,116,afternoon,thursday,weekend,unknown,paid,sunny,Palestine,434,0
2,225,354,noon,saturday,workday,unknown,free,stormy,Hungary,347,0
3,108,5,sunset,wednesday,workday,home,free,drizzle,Japan,391,0
4,257,33,morning,thursday,workday,unknown,free,cloudy,United States,0,1
...,...,...,...,...,...,...,...,...,...,...,...
288604,133,28,evening,friday,weekend,unknown,free,snowy,Finland,1009,0
288605,583,2445,afternoon,friday,workday,unknown,free,unknown,unknown,0,1
288606,264,16,sunrise,tuesday,weekend,home,free,snowy,Lebanon,306,0
288607,70,336,sunrise,tuesday,workday,unknown,free,sleet,Japan,919,0


In [8]:
labels = df['label']
df.drop(columns=['label'], inplace=True)

In [9]:
df.columns

Index(['user', 'item', 'daytime', 'weekday', 'isweekend', 'homework', 'cost',
       'weather', 'country', 'city'],
      dtype='object')

In [10]:
sparse_columns = ['user', 'item', 'daytime', 'weekday', 'isweekend', 'homework', 'cost',
       'weather', 'country', 'city']

dense_columns = []

In [11]:
sparse_encoders = {}
for column in sparse_columns:
    sparse_encoders[column] = LabelEncoder()
    df[column] = sparse_encoders[column].fit_transform(df[column].values)

In [12]:
dense_encoders = {}
for column in dense_columns:
    dense_encoders[column] = StandardScaler()
    df[column] = dense_encoders[column].fit_transform(df[column].values.reshape(-1,1))

In [13]:
features = df.nunique().to_dict()

In [14]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)


In [15]:
def loss_function(y_true, y_pred):
    
    loss = loss_object(y_true=y_true, y_pred=y_pred)
    
    rmse = tf.math.sqrt(loss)
    
    return rmse

In [23]:
adfm = ADFM.ADFM(features, sparse_columns, dense_columns, embedding_size=256, attention_factor=16, rate=0.3)


In [24]:
x_train, x_test, y_train, y_test = train_test_split(df, labels, test_size=0.1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [25]:
train_loss = tf.keras.metrics.BinaryCrossentropy(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
train_auc = tf.keras.metrics.AUC(name='train_auc')

val_loss = tf.keras.metrics.BinaryCrossentropy(name='val_loss')
val_accuracy = tf.keras.metrics.BinaryAccuracy(name='val_accuracy')
val_auc = tf.keras.metrics.AUC(name='val_auc')

test_loss = tf.keras.metrics.BinaryCrossentropy(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')
test_auc = tf.keras.metrics.AUC(name='test_auc')



In [26]:
def train_step(x, target):

    with tf.GradientTape() as tape:
        predictions  = adfm(x, True)
        loss = loss_function(y_true=target, y_pred=predictions)
                
    gradients = tape.gradient(loss, adfm.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, adfm.trainable_variables))

    train_loss(y_true=target, y_pred=predictions)
    train_accuracy(y_true=target, y_pred=predictions)
    train_auc(y_true=target, y_pred=predictions)


In [27]:
EPOCHS = 5
BATCH_SIZE = 128
STEPS = x_train.shape[0] // BATCH_SIZE

In [28]:
checkpoint_path = "./checkpoints/frappe/ADFM"

ckpt = tf.train.Checkpoint(model=adfm,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

In [29]:
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()
    train_auc.reset_states()

    
    val_loss.reset_states()
    val_accuracy.reset_states()
    val_auc.reset_states()

    for batch in range(STEPS):
        
        sample = x_train.sample(n=BATCH_SIZE)
        indexs = sample.index
        y = y_train[indexs].values.reshape((-1,1))
        
        train_step(sample, y)

        if batch % 400 == 0:
            print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
              epoch + 1, batch, train_loss.result(), train_accuracy.result(), train_auc.result()))

    for batch in range(x_val.shape[0] // BATCH_SIZE):

        sample = x_val.sample(n=BATCH_SIZE)
        indexs = sample.index
        y = y_val[indexs].values.reshape((-1,1))
        sparse_x = {k: np.array(list(v.values())) for k, v in sample[sparse_columns].to_dict().items()}
        dense_x = {k: np.array(list(v.values())) for k, v in sample[dense_columns].to_dict().items()}
        val_predictions = adfm(sample, False)

        val_loss(y_true=y, y_pred=val_predictions)
        val_accuracy(y_true=y, y_pred=val_predictions)
        val_auc(y_true=y, y_pred=val_predictions)

    print()
    print('Validation Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
      val_loss.result(), val_accuracy.result(), val_auc.result()))
    print()

    
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                             ckpt_save_path))

        print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                    train_loss.result(), 
                                                    train_accuracy.result()))

        print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))


Epoch 1 Batch 0 Loss 0.6791 Accuracy 0.6406 AUC 0.5887


KeyboardInterrupt: 

In [ ]:
test_loss.reset_states()
test_accuracy.reset_states()
test_auc.reset_states()

for batch in range(x_test.shape[0] // BATCH_SIZE):

        sample = x_test.sample(n=BATCH_SIZE)
        indexs = sample.index
        y = y_test[indexs].values.reshape((-1,1))
        test_predictions = deepAfm(sample, False)

        test_loss(y_true=y, y_pred=test_predictions)
        test_accuracy(y_true=y, y_pred=test_predictions)
        test_auc(y_true=y, y_pred=test_predictions)
        
print('Test Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
      test_loss.result(), test_accuracy.result(), test_auc.result()))